### 필요한것들 다운 (모듈, 크롬옵션)

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin
!pip install webdriver_manager
!pip install chromedriver_autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadco

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat
from itertools import groupby

import pandas as pd
import numpy as np

import pickle

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # GUI 없는 환경에서는 headless 모드를 사용
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

### 최종 코드(함수)

i : 한국 / 해외 차트 선택
  - 1:한국, 2:해외

j : 장르
  - 1 ~ 8까지 바뀜

k : 스테디셀러 클릭
  - 4, 6 왔다갔다
  - 세부 장르가 있으면 6, 없으면 4

In [4]:
def get_songs_kor(i,j,k): # i:한국/해외, j:장르, k:스테디셀러

    driver = webdriver.Chrome(options = chrome_options)
    driver.get('https://www.melon.com/genre/song_list.htm?gnrCode=GN0100')
    time.sleep(5)

  ## -------[한국/해외 - 장르 - 스테디셀러 선택]----------------------------------------------------

    # 한국(1)/해외(2) 선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="gnb_menu"]/ul[1]/li[3]/div/ul/li[{i}]/a'))).click()

    # 장르 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="conts"]/div[2]/ul/li[{j}]/a'))).click()

    # 스테디셀러 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="conts"]/div[{k}]/ul/li[3]/a'))).click()

    # 페이지 로드 기다리기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((
            By.XPATH, '//*[@id="frm"]/div/div/button[1]')))

    # 어떤 장르인지 저장
    genre = driver.find_element(By.XPATH, f'//*[@id="conts"]/div[2]/ul/li[{j}]/a/span').text

  ## -------[노래 특성 가져오기]----------------------------------------------------

    # 총 페이지 수
    pages = len(driver.find_element(By.XPATH, '//*[@id="pageObjNavgation"]/div/span').text)

    song_num = []

    for page in range(1, pages+1):

      # 페이지 잘 넘어가는지 체크하기 위함
      # print(driver.find_element(By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[2]/div/span[1]').text)

      if page < pages:

        for song in range(1,51):

          link = driver.find_element(By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{song}]/td[7]/div/button')

          song_num.append(link.get_attribute('data-song-no'))

      else:

        for song in range(1,len(driver.find_elements(By.CLASS_NAME, 'rank'))):

          link = driver.find_element(By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{song}]/td[7]/div/button')

          song_num.append(link.get_attribute('data-song-no'))

      driver.execute_script(f"pageObj.sendPage({page*50 + 1});")

      WebDriverWait(driver,10).until(EC.presence_of_element_located((
          By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[2]/div/span[1]')))

      time.sleep(6)

    print(f"총 {len(song_num)}곡의 노래를 가져옵니다!")
    # print(song_num)

      # 노래 번호 가져오기 완료 !

    titles = []
    artists = []
    lyrics = []
    whens = []
    likes = []

    for index in range(len(song_num)):

      number = song_num[index]

      # driver = webdriver.Chrome(options = chrome_options)
      driver.get(f'https://www.melon.com/song/detail.htm?songId={number}') # 새로운 페이지 오픈

      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

      title = driver.find_element(By.CLASS_NAME, 'song_name').text
      artist = driver.find_element(By.CLASS_NAME, 'artist').text

      # 가사 없는 경우 np.nan으로 처리
      lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
      lyric =  lyric_element[0].text if lyric_element else np.nan

      when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
      like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text

      titles.append(title)
      artists.append(artist)
      lyrics.append(lyric)
      whens.append(when)
      likes.append(like)

      if index != 0 and index % 50 == 0:
        print("한 페이지 완료!")

      if index == len(song_num) - 1:
        print("노래 가져오기 끝!")

    genre_rep = np.repeat(genre, len(titles))

    df = pd.DataFrame({'장르':genre_rep, '제목':titles, '가수':artists, '가사':lyrics,
                      '발매년도':whens, '좋아요 수':likes})

    genre_clean = genre.replace('/', '-')

    df.to_pickle(f'/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 5 - 데이터 수집/DATA/Genres/pickles/한국_{genre_clean}.pkl')

    return df

In [17]:
def get_songs_pop(i,j,k): # i:한국/해외, j:장르, k:스테디셀러

    driver = webdriver.Chrome(options = chrome_options)
    driver.get('https://www.melon.com/genre/song_list.htm?gnrCode=GN0100')
    time.sleep(5)

  ## -------[한국/해외 - 장르 - 스테디셀러 선택]----------------------------------------------------

    # 한국(1)/해외(2) 선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="gnb_menu"]/ul[1]/li[3]/div/ul/li[{i}]/a'))).click()

    # 장르 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="conts"]/div[2]/ul/li[{j}]/a'))).click()

    # 스테디셀러 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="conts"]/div[{k}]/ul/li[3]/a'))).click()

    # 페이지 로드 기다리기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((
            By.XPATH, '//*[@id="frm"]/div/div/button[1]')))

    # 어떤 장르인지 저장
    genre = driver.find_element(By.XPATH, f'//*[@id="conts"]/div[2]/ul/li[{j}]/a/span').text

  ## -------[노래 특성 가져오기]----------------------------------------------------

    # 총 페이지 수
    pages = len(driver.find_element(By.XPATH, '//*[@id="pageObjNavgation"]/div/span').text)

    song_num = []

    for page in range(1, pages+1):

      # 페이지 잘 넘어가는지 체크하기 위함
      # print(driver.find_element(By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[2]/div/span[1]').text)

      if page < pages:

        for song in range(1,51):

          link = driver.find_element(By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{song}]/td[7]/div/button')

          song_num.append(link.get_attribute('data-song-no'))

      else:

        for song in range(1,len(driver.find_elements(By.CLASS_NAME, 'rank'))):

          link = driver.find_element(By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{song}]/td[7]/div/button')

          song_num.append(link.get_attribute('data-song-no'))

      driver.execute_script(f"pageObj.sendPage({page*50 + 1});")

      WebDriverWait(driver,10).until(EC.presence_of_element_located((
          By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[2]/div/span[1]')))

      time.sleep(6)

    print(f"총 {len(song_num)}곡의 노래를 가져옵니다!")
    print(song_num)

      # 노래 번호 가져오기 완료 !

    titles = []
    artists = []
    lyrics = []
    whens = []
    likes = []

    for index in range(len(song_num)):

      number = song_num[index]

      driver = webdriver.Chrome(options = chrome_options)
      driver.get(f'https://www.melon.com/song/detail.htm?songId={number}') # 새로운 페이지 오픈

      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

      title = driver.find_element(By.CLASS_NAME, 'song_name').text
      artist = driver.find_element(By.CLASS_NAME, 'artist').text

      # 가사 없는 경우 np.nan으로 처리
      lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
      lyric =  lyric_element[0].text if lyric_element else np.nan

      when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
      like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text

      titles.append(title)
      artists.append(artist)
      lyrics.append(lyric)
      whens.append(when)
      likes.append(like)

      if index != 0 and index % 50 == 0:
        print("한 페이지 완료!")

      if index == len(song_num) - 1:
        print("노래 가져오기 끝!")

    genre_rep = np.repeat(genre, len(titles))

    df = pd.DataFrame({'장르':genre_rep, '제목':titles, '가수':artists, '가사':lyrics,
                      '발매년도':whens, '좋아요 수':likes})

    genre_clean = genre.replace('/', '-')

    df.to_pickle(f'/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 5 - 데이터 수집/DATA/Genres/pickles/해외_{genre_clean}.pkl')

    return df

### 한국대중음악

발라드, 댄스, 랩/힙합, R&B/Soul
- i : 1
- j : 1 ~ 4
- k : 4

최종 - 349, 415, 264, 262 => 1290

In [6]:
i = 1
k = 4

#### 발라드 - 완료 !!!
# get_songs_kor(i, 1, k)
# print("완료했습니당 :)")

#### 댄스 - 완료 !!!
# get_songs_kor(i, 2, k) # 415곡에 20분
# print("완료했습니당 :)")

#### 랩/힙합 - 완료 !!!
# get_songs_kor(i, 3, k)
# print("완료했습니당 :)")

#### R&B/Soul - 완료 !!!
# get_songs_kor(i, 4, k) # 262곡에 15분
# print("완료했습니당 :)")


총 349곡의 노래를 가져옵니다!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
노래 가져오기 끝!
완료했습니당 :)


인디음악
- i : 1
- j : 5
- k : 6

최종 - 1290 + 156 = 1446

In [ ]:
#### 인디음악 - 완료 !!!
# get_songs_kor(1,5,6)
# print("완료했습니당 :)")

록/메탈
- i : 1
- j : 6
- k : 4

최종 - 1446 + 380 = 1826

In [11]:
#### 록/메탈 - 완료 !!!
# df = get_songs_kor(1,6,4)
# print("완료했습니당 :)")
# df.head()

총 380곡의 노래를 가져옵니다!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
한 페이지 완료!
노래 가져오기 끝!
완료했습니당 :)


,장르,제목,가수,가사,발매년도,좋아요 수
0,록/메탈,벚꽃 엔딩,버스커 버스커,그대여 그대여 그대여\n그대여 그대여\n오늘은 우리 같이 걸어요 이 거리를\n밤에 ...,2012.03.29,"320,051"
1,록/메탈,"스물다섯, 스물하나",자우림,바람에 날려 꽃이 지는 계절엔\n아직도 너의 손을 잡은 듯 그런 듯 해\n그때는 아...,2013.10.14,"232,067"
2,록/메탈,사랑했나봐,윤도현,이별은 만남보다 참 쉬운 건가봐\n차갑기만 한 사람\n내 맘 다 가져간걸 왜 알지 ...,2005.04.29,"145,225"
3,록/메탈,기억을 걷는 시간,넬 (NELL),아직도 너의 소리를 듣고\n아직도 너의 손길을 느껴\n오늘도 난 너의 흔적 안에 살...,2008.03.21,"54,468"
4,록/메탈,남자를 몰라,버즈,매번 늦어도 이해 할게\n누굴 만났니\n먼저 묻지 않을게\n고집스런 내 사랑\n너의...,2006.04.24,"110,597"


트로트
- i : 1
- j : 7
- k : 6

최종 - 1826 + 272 = 2098

In [ ]:
#### 트로트 - 롼료 !!!!
# df = get_songs_kor(1,7,6)
# print("완료했습니당 :)")
# df.head()

포크/블루스
- i : 1
- j : 8
- k : 4

최종 - 2098 + 216 = 2314

In [ ]:
#### 포크/블루스 - 완료 !!!!
# df = get_songs_kor(1,8,4)
# print("완료했습니당 :)")
# df.head()

### 해외POP음악

POP, 록/메탈, 일렉트로니카, 랩/힙합, R&B/Soul, 포크/블루스/컨트리

- i : 2
- j : 1 ~ 6
- k : 6

최종 - 646

In [21]:
i = 2
k = 6

### 완료 ~~

# df1 = get_songs_pop(i, 1, k)
# print("완료했습니당 :)")

# df2 = get_songs_pop(i, 2, k)
# print("완료했습니당 :)")

# df3 = get_songs_pop(i, 3, k)
# print("완료했습니당 :)")

# df4 = get_songs_pop(i, 4, k)
# print("완료했습니당 :)")

# df5 = get_songs_pop(i, 5, k)
# print("완료했습니당 :)")

# df6 = get_songs_pop(i, 6, k)
# print("완료했습니당 :)")

총 117곡의 노래를 가져옵니다!
['3986050', '3986048', '4660792', '3808189', '3501861', '5800348', '7968702', '4660799', '1922885', '3808190', '3808188', '5834284', '5718697', '3143269', '1658609', '5834282', '834030', '290109', '3143268', '4358378', '3501869', '5718699', '893571', '5401522', '2043750', '2183950', '2756660', '1479147', '5391781', '497113', '1929312', '3808186', '7915317', '4626885', '5544182', '393299', '2108423', '3808192', '7866026', '11111604', '240309', '470152', '771913', '5767245', '2367655', '3986046', '4371373', '771908', '1274333', '4660793', '3808182', '285963', '3971177', '3808183', '395876', '5830416', '346917', '290111', '3808191', '2332028', '1590592', '5402216', '3986045', '5790011', '5760114', '497106', '4399665', '1781178', '2177582', '6453239', '3143267', '648768', '3143273', '1144241', '1658614', '333020', '4491979', '3808185', '3069949', '5767249', '7964945', '4254725', '2510494', '5488594', '5488585', '5751727', '515486', '5816814', '1600134', '3808187', '36837

In [23]:
df6.head()

,장르,제목,가수,가사,발매년도,좋아요 수
0,포크/블루스/컨트리,"You've Got a Friend in Me (From ""Toy Story""/So...",Randy Newman,You've got a friend in me\nWhen the road looks...,1995.01.01,"41,594"
1,포크/블루스/컨트리,The Blower's Daughter,Damien Rice,And so it is\njust like you said it would be\n...,2003.08.26,"52,132"
2,포크/블루스/컨트리,New York City,"Norah Jones, The Peter Malick Group",I can't remember\nwhat I planned tomorrow\nI c...,2003.07.08,"24,893"
3,포크/블루스/컨트리,Cayman Islands,Kings Of Convenience,through the alleyways\nto cool off in the shad...,2004.01.01,"33,576"
4,포크/블루스/컨트리,Mrs. Cold,Kings Of Convenience,Hey baby Mrs Cold\nActing so tough\nDidn't kno...,2009.01.01,"13,096"


### 이것저것

In [ ]:
driver = webdriver.Chrome(options = chrome_options)
driver.get('https://www.melon.com/song/detail.htm?songId=36859858') # 새로운 페이지 오픈

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

title = driver.find_element(By.CLASS_NAME, 'song_name').text
artist = driver.find_element(By.CLASS_NAME, 'artist').text

# 가사 없는 경우 np.nan으로 처리
lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
lyric =  lyric_element[0].text if lyric_element else np.nan

when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text

print(lyric)

枯れていく
카레테이쿠
시들어 가는

今この瞬間も
이마 코노 순칸모
지금 이 순간에도

咲いている
사이테이루
피어 있는

全ては溶けていく
스베테와 토케테이쿠
모든 건 녹아가네

何が出来るのだろうか
나니가 데키루노다로오카
무얼 할 수 있을까

誰を生きょうかな
다레오 이키요오카나
누구로 살아가야 할까

みんな儚い
민나 하카나이
모두 허무하고

みんな無い
민나 토오토이
모두 소중하지

しわしわに萎れた花束
시와시와니 시오레타 하나타바
시들시들 시들어버린 꽃다발

小わきに抱えて
코와키니 카카에테
품에 안고서

永遠に変わらぬ輝き
에이엔니 카와라누 카가야키
영원히 변치 않을 광채를

探してた
사가시테타
찾아 헤맸지

僕らを信じてみた
보쿠라오 신지테 미타
우리들을 믿어봤어

僕らを感じてた
보쿠라오 칸지테타
우리들을 느껴봤어

咲かせにいくよ
사카세니 이쿠요
피워내러 가볼게

内なる花を
우치나루 하나오
내면의 꽃을

さりげなく
사리게나쿠
아무렇지 않게

思いを込めてみる
오모이오 코메테미루
마음을 담아 보고

やむを得ず
야무오에즈
어쩔 수 없이

所りを込めていく
이노리오 코메테이쿠
기도를 담고 있네

いつまで迷うんだろうか
이츠마데 마요운다로오카
언제까지 망설이는 걸까

いつかは分かるよな
이츠카와 와카루요나
언젠가는 알게 되겠지

誰もが一人
다레모가 히토리
모두가 혼자

全ては一つ
스베테와 히토츠
모든 건 하나

色々な姿や形に
이로이로나 스가타야 카타치니
이런저런 모습과 형태에

惑わされるけど
마도와사레루케도
현혹되기도 하지만

いつの日か
이츠노 히카
언젠가는

全てがかわいく思えるさ
스베테가 카와이쿠 오모에루사
모든 게 사랑스러워 보일 거야

わたしは何になろうか
와타시와 나니니나로오카
나는 무엇이 되어 볼까

どんな色がいいかな
돈나 이로가 이이카나
어떤 색이 좋을까

探しにいくよ
사가시니 이쿠요
찾으러 가볼게

内なる花を
우치나루 하나오
내면의 꽃을

しわしわに萎れた花束
시와시와니 시오레타 하나타바
시들시들 시들어버린 꽃다발

小わきに抱えて
코와키니 카카에테
품에 안고서

永遠に変わ